# Potential Speedup imporvements

path generation
* remove any use of ROUND()
* while returning paths, make sure all nodes in the paths are network nodes (no origin or destination nodes..).

subgraph generation
* consoliate + weight

destination discovery
* stop searching for origin nodes past 0.5 search radius
* remove duplicate if statements


update light graph
* shorter list of segments to iterate over
* one add one remove


betweenness
* remove decay calculations from inner loop.
* vectorize betweenness calculation outside path loop
* reduce calls to node_gdf.at[] by doing them when thwey're first possible, instead of inside path loop
* vectorize destination probability calculations



path generation
* return paths as edges
* remove len(neighbors) = 1 check, causes excessive calls to neighbors.
* consolidate edge information retreval from graph.

betweenness
* eliminate edge list construction inside loop.


# For next week:
* Send rounaq wide table of counts
* Once recieved, from lui, start a new digitization process in the new network
* look at th eparks file situation

# Functionality List 3
* set up a slideshow/diagram to show the relationship between different compoonents of the library.
* Set up a notebook to go over estimating flows from one origin to one destination, Document all relevant settings
* show how this generalizes to the pairings.csv, iterating over moodel settings, iterating over scenarios
* Make a finalized github commit including testing script, betweenness flow notebook. 
* Deploy to workstation.
* given the simulation results, and the censor locations, find a way to calibrate counts and generate beta coeffecients (Beirut, SOmerville, NYC)
* enable Jupyter notebook server access from workstation.
* Figure out a way to do the "Cities" dropbox folder, people can add data through dropbox, run simulations through Jupyter Server, get results through Dropbox

## Map Cleaning
https://www.dropbox.com/scl/fo/d6ugcyt3aanfb3yk4n9va/h?dl=0&rlkey=1c2giilvzoyygcxrlry7fscr9



## Needed compatibility upgrades:
* The query_bulk() method of the spatial index .sindex property is deprecated in favor of query() (#2823).
* Make surr there is no direct use of pyGEOS (currently used in node-edge creation if tolerance=0) [ If you use PyGEOS directly and access an array of PyGEOS geometries using GeoSeries.values.data, you will need to make some changes to avoid code breakage.]  , a simple replacement of pygeos with shapely together with a change of gdf.geometry.values.data to gdf.geometry.values or analogous gdf.geometry.array should work


## instaling an environment
```
Run Powershell as administrator

conda create --name=madina_0_0_2 python
conda activate madina_0_0_2
conda config --env --add channels conda-forge
conda config --env --set channel_priority strict
conda install -f mkl
conda install python=3 geopandas
conda install pydeck
conda install pyogrio
conda install rtrees
```

In [ ]:
%config Completer.use_jedi = False

import sys
sys.path.append('../')

from madina.zonal.zonal import Zonal
from madina.una.betweenness import parallel_betweenness, paralell_betweenness_exposure, betweenness_exposure


import pandas as pd
import geopandas as gpd
import numpy as np
import time 



print (f"{'test name':30s} | {'madina_flow_sum':15s} | {'Rhino_flow_sum':15s} | {'sum_diff':8s} | {'sum_smlr_%':10s} | {'sle_mean':8s} | {'sle_median':10s} | {'sle_max':8s} | {'sle>0.1%':8s} | {'sle>1.0%':8s} | {'sle>3.0%':8s} | {'sle>5.0%':8s} | {'time_spent':10s}")
#for test_case in os.listdir("Test Cases"):
for test_case in ['Harvard Square', 'Somerville']:   # 'Harvard Square',
    start = time.time()
    # TODO: Check OS compatibility, ensure this is compatible with Unix systems..
    test_case_folder = "Test Cases" + "\\" + test_case + "\\"
    test_config = pd.read_csv(test_case_folder + "test_configs.csv")
    test_flows =  pd.read_csv(test_case_folder + "test_flows.csv")

    if test_case == 'Somerville':
        ready_tests = test_config.index[0:3]
    else:
        ready_tests = test_config.index

    for test_idx in ready_tests:

        harvard_square = Zonal()

        harvard_square.load_layer(
            layer_name='streets',
            file_path=  test_case_folder + test_config.at[test_idx, 'Network_File']
            )

        harvard_square.load_layer(
            layer_name=test_config.at[test_idx, 'Origin_Name'],
            file_path= test_case_folder + test_config.at[test_idx, 'Origin_File']
            )

        harvard_square.load_layer(
            layer_name=test_config.at[test_idx, 'Destination_Name'],
            file_path= test_case_folder + test_config.at[test_idx, 'Destination_File']
            )
        

        harvard_square.create_street_network(
            source_layer='streets', 
            discard_redundant_edges=True,
            split_redundant_edges=False,
            node_snapping_tolerance=0.1,  # TODO: check for sensitivity... pick one as default snapping.
            weight_attribute=test_config.at[test_idx, 'Network_Cost'] if test_config.at[test_idx, 'Network_Cost'] != "Geometric" else None
        )

        #print (f"{harvard_square.network.edges.shape = }")
        #print (f"{harvard_square.network.nodes.shape = }")
        #harvard_square.network.nodes,  harvard_square.network.edges = _discard_redundant_edges(harvard_square.network.nodes, harvard_square.network.edges)
        #print (f"{harvard_square.network.edges.shape = }")
        #print (f"{harvard_square.network.nodes.shape = }")

        harvard_square.insert_node(
            layer_name=test_config.at[test_idx, 'Origin_Name'], 
            label='origin', 
            weight_attribute=test_config.at[test_idx, 'Origin_Weight'] if test_config.at[test_idx, 'Origin_Weight'] != "Count" else None
        )
        harvard_square.insert_node(
            layer_name=test_config.at[test_idx, 'Destination_Name'], 
            label='destination', 
            weight_attribute=test_config.at[test_idx, 'Destination_Weight'] if test_config.at[test_idx, 'Destination_Weight'] != "Count" else None
        )

        harvard_square.create_graph()

        node_gdf = harvard_square.network.nodes
        origin_gdf = node_gdf[node_gdf['type'] == 'origin']

        harvard_square.network.nodes["original_weight"] = harvard_square.network.nodes["weight"]

        harvard_square.network.turn_penalty_amount = test_config.at[test_idx, 'Turn_Penalty']
        harvard_square.network.turn_threshold_degree = test_config.at[test_idx, 'Turn_Threshold']

        if test_config.at[test_idx, 'Elastic_Weights']:
            '''
            harvard_square.network.nodes["weight"] = harvard_square.network.nodes["original_weight"]
            get_elastic_weight(
                harvard_square.network,
                search_radius=test_config.at[test_idx, 'Radius'],
                detour_ratio=test_config.at[test_idx, 'Detour'],
                beta=test_config.at[test_idx, 'Beta'],
                decay=True, #test_config.at[test_idx, 'Decay'],
                #turn_penalty=test_config.at[test_idx, 'Turns'],
                turn_penalty=False,
            )
            for o_idx in origin_gdf.index:
                harvard_square.network.nodes.at[o_idx, 'weight'] =  harvard_square.network.nodes.at[o_idx, 'elastic_weight']
            '''
            continue
        '''
        return_dict = parallel_betweenness(
            harvard_square.network,
            search_radius=test_config.at[test_idx, 'Radius'],
            detour_ratio=test_config.at[test_idx, 'Detour'],
            decay=test_config.at[test_idx, 'Decay'], #if test['Elastic weights'] else True,
            decay_method=test_config.at[test_idx, 'Decay_Mode'],  # "power", "exponent"
            beta=test_config.at[test_idx, 'Beta'],
            path_detour_penalty='equal', # "equal",  # "power", "exponent", "equal"
            origin_weights=False if type(test_config.at[test_idx, 'Origin_Weight']) != str else True,
            closest_destination=test_config.at[test_idx, 'Closest_Destination'],
            destination_weights=False if type(test_config.at[test_idx, 'Destination_Weight']) != str  else True,    #or (test['Elastic weights'])
            # perceived_distance=False,
            num_cores=6,
            light_graph=True,
            turn_penalty=test_config.at[test_idx, 'Turns'],
        )
        
        '''
        return_dict = paralell_betweenness_exposure(
            harvard_square,
            search_radius=test_config.at[test_idx,'Radius'],
            detour_ratio=test_config.at[test_idx,'Detour'],
            decay=False if test_config.at[test_idx,'Elastic_Weights'] else test_config.at[test_idx,'Decay'],  # elastic weight already reduces origin weight factoring in decay. if this pairing uses elastic weights, don't decay again,
            decay_method=test_config.at[test_idx,'Decay_Mode'],
            beta=test_config.at[test_idx,'Beta'],
            num_cores=3,
            path_detour_penalty='equal', # "power" | "exponent" | "equal"
            closest_destination=test_config.at[test_idx,'Closest_Destination'],
            elastic_weight=test_config.at[test_idx,'Elastic_Weights'],
            turn_penalty=test_config.at[test_idx,'Turns'],
            path_exposure_attribute=None,
            return_path_record=False, 
            destniation_cap=None
        )

        



        simulated_sum_of_flow = return_dict['edge_gdf']['betweenness'].sum()
        test_flow = test_flows[test_config.at[test_idx, 'Flow_Name']].sum()


        ## create segment level comparison
        # creating connector lines

        import shapely.geometry as geo
        simulated_betweenness = return_dict['edge_gdf'][['betweenness', 'parent_street_id']].rename(columns={"betweenness": "simulated_betweenness"}).drop_duplicates(subset=['parent_street_id']).set_index("parent_street_id")
        simulated_betweenness = harvard_square.layers["streets"].gdf[["geometry", "__GUID"]].join(simulated_betweenness).set_index("__GUID")

        test_name = test_config.at[test_idx, 'Flow_Name']
        test_betweenness = test_flows[['__GUID', test_name]].set_index("__GUID").rename(columns = {test_name: "test_flow"})


        comparison = simulated_betweenness.join(test_betweenness)
        comparison["difference"] = comparison["simulated_betweenness"] - comparison["test_flow"]
        comparison["difference_pct"] = abs(comparison["difference"]) / comparison["simulated_betweenness"] *100
        # segment level error
        sle = comparison[~comparison["difference_pct"].isna()]['difference_pct']

        
        node_gdf = harvard_square.network.nodes
        edge_gdf = harvard_square.network.edges


        origin_layer = harvard_square.layers[test_config.at[test_idx, 'Origin_Name']].gdf
        origin_nodes = node_gdf[node_gdf['type'] == 'origin']
        origin_joined = origin_layer.join(origin_nodes.set_index('source_id'),lsuffix='_origin')
        origin_joined['connector_line'] = origin_joined.apply(lambda x:geo.LineString([x['geometry'], x["geometry_origin"]]), axis=1)
        origin_joined["geometry"] = origin_joined['connector_line']


        destination_layer = harvard_square.layers[test_config.at[test_idx, 'Destination_Name']].gdf
        destination_nodes = node_gdf[node_gdf['type'] == 'destination']
        destination_joined = destination_layer.join(destination_nodes.set_index('source_id'),lsuffix='_destination')
        destination_joined['connector_line'] = destination_joined.apply(lambda x:geo.LineString([x['geometry'], x["geometry_destination"]]), axis=1)
        destination_joined["geometry"] = destination_joined['connector_line']


        #print (f"{origin_nodes.shape = }\t{destination_nodes.shape = }")

        streets = harvard_square.layers["streets"].gdf 
        network_file = gpd.read_file(test_case_folder + test_config.at[test_idx, 'Network_File'], engine='pyogrio')


        flow_difference = comparison[
            ((comparison['test_flow' ] > 0) & (comparison['simulated_betweenness'] == 0)) | 
            ((comparison['test_flow' ] == 0) & (comparison['simulated_betweenness'] > 0))
        ]
        '''
        harvard_square.create_map(
            [
                #{'gdf': streets[streets[test_config.at[test_idx, 'Network_Cost']] > 0], 'color': [255, 255, 255], 'text': test_config.at[test_idx, 'Network_Cost']},
                {'gdf': streets, 'color': [100, 100, 100], 'opacity': 0.1},
                {'gdf': edge_gdf[edge_gdf['betweenness'] > 0], 'color': ['125, 125, 0'], 'text': 'betweenness', 'opacity': 0.2},
                #{'gdf': comparison[abs(comparison["difference"]) > 0.01], 'color_by_attribute': 'difference', 'color_method': 'gradient', 'text': 'difference'},
                {'gdf': comparison[(comparison["difference_pct"] >= 0.1) & (comparison["difference_pct"] < 100)], 'color_by_attribute': 'difference_pct', 'color_method': 'gradient', 'text': 'difference_pct'},
                {'gdf': edge_gdf[edge_gdf['snapped'] == True], 'color': [255, 0, 255], 'opacity': 0.2},
                {'gdf': network_file[network_file["geometry"].geom_type == 'Polygon'], 'color': [125, 0, 125], 'text': '__GUID'},
                {'gdf': flow_difference, 'color': [255, 255, 0] , 'text': 'difference'},        
                #{'gdf': comparison[comparison['difference'] != 0], 'color_by_attribute': 'difference', 'color_method': 'gradient', 'text': 'difference', 'opacity':  0.1},
                {'gdf': origin_layer, 'color': [0, 0, 255]},
                {'gdf': origin_joined[['geometry']], 'color': [0, 0, 255]},
                {'gdf': destination_layer, 'color': [255, 0, 0]},
                {'gdf': destination_joined[['geometry']], 'color': [255, 0, 0]},
                {'gdf': harvard_square.network.nodes[['geometry', 'type', 'weight']].reset_index(), 'color': [255, 0, 255], 'text': 'id'},
            ],
            save_as="Test Cases\\" + test_case + '\\'  + test_name + "._difference_map.html"
        )
        '''
        
        #print (test_config.loc[test_idx])
        print (f"{test_config.at[test_idx, 'Flow_Name'][:30]:30s} | {simulated_sum_of_flow:15.2f} | {test_flow:15.2f} | {simulated_sum_of_flow - test_flow:8.2f} | {1-(simulated_sum_of_flow - test_flow)/ test_flow:10.2%} | {sle.mean():7.4f}% | {sle.median():9.4f}% | {sle.max():7.4f}% | {sle[sle > 0.1].count():8} | {sle[sle > 1.0].count():8} | {sle[sle > 3.0].count():8} | {sle[sle > 5.0].count():8} | {time.time() - start: 10.5f}")
        #print ("DOne Case...")
        #break
    #break

In [ ]:
## vectorized exposure betweennes swith segment ids, removing len 1 neighbor check, consolidating edge information retreval
test name                      | madina_flow_sum | Rhino_flow_sum  | sum_diff | sum_smlr_% | sle_mean | sle_median | sle_max  | sle>0.1% | sle>1.0% | sle>3.0% | sle>5.0% | time_spent
_Betweenness1                  |          702.07 |          702.07 |    -0.00 |    100.00% |  0.0000% |    0.0000% |  0.0000% |        0 |        0 |        0 |        0 |    3.87253
_Betweenness2                  |        19025.63 |        19025.63 |     0.00 |    100.00% |  0.0000% |    0.0000% |  0.0000% |        0 |        0 |        0 |        0 |    6.86009
_Betweenness3                  |        19214.00 |        19214.00 |     0.00 |    100.00% |  0.0000% |    0.0000% |  0.0000% |        0 |        0 |        0 |        0 |    9.66996
_Betweenness4                  |        20853.18 |        20853.18 |    -0.00 |    100.00% |  0.0000% |    0.0000% |  0.0000% |        0 |        0 |        0 |        0 |   12.36717
Somerville_Bus_Subway_Geometri |          700.29 |          713.14 |   -12.84 |    101.80% |  2.3488% |    2.2893% |  5.0852% |      266 |      232 |       96 |        2 |   16.73959
Somerville_Bus_Subway          |          730.95 |          743.95 |   -13.00 |    101.75% |  2.4117% |    2.2917% |  5.1912% |      285 |      251 |      103 |        8 |   34.42406
Somerville_Homes_Subway        |       368161.49 |       389383.31 | -21221.81 |    105.45% |     inf% |    7.2526% |     inf% |     2665 |     2631 |     2521 |     2216 |  319.56011
## vectorized exposure betweennes swith segment ids
test name                      | madina_flow_sum | Rhino_flow_sum  | sum_diff | sum_smlr_% | sle_mean | sle_median | sle_max  | sle>0.1% | sle>1.0% | sle>3.0% | sle>5.0% | time_spent
_Betweenness1                  |          702.07 |          702.07 |    -0.00 |    100.00% |  0.0000% |    0.0000% |  0.0000% |        0 |        0 |        0 |        0 |    3.75356
_Betweenness2                  |        19025.63 |        19025.63 |     0.00 |    100.00% |  0.0000% |    0.0000% |  0.0000% |        0 |        0 |        0 |        0 |    6.62444
_Betweenness3                  |        19214.00 |        19214.00 |     0.00 |    100.00% |  0.0000% |    0.0000% |  0.0000% |        0 |        0 |        0 |        0 |   10.33437
_Betweenness4                  |        20853.18 |        20853.18 |    -0.00 |    100.00% |  0.0000% |    0.0000% |  0.0000% |        0 |        0 |        0 |        0 |   14.16261
omerville_Bus_Subway_Geometri |          700.29 |          713.14 |   -12.84 |    101.80% |  2.3488% |    2.2893% |  5.0852% |      266 |      232 |       96 |        2 |   18.64562
Somerville_Bus_Subway          |          730.95 |          743.95 |   -13.00 |    101.75% |  2.4117% |    2.   2917% |  5.1912% |      285 |      251 |      103 |        8 |   37.69253
Somerville_Homes_Subway        |       368161.49 |       389383.31 | -21221.81 |    105.45% |     inf% |    7.2526% |     inf% |     2665 |     2631 |     2521 |     2216 |  324.89220



# One betweenness, with all the new network fixes... (THIS IS ON ^ CORES> DO NOT COMPARE TIME)
test name                      | madina_flow_sum | Rhino_flow_sum  | sum_diff | sum_smlr_% | sle_mean | sle_median | sle_max  | sle>0.1% | sle>1.0% | sle>3.0% | sle>5.0% | time_spent
_Betweenness1                  |          704.85 |          702.07 |     2.78 |     99.60% |  0.0376% |    0.0000% |  2.4893% |        2 |        2 |        0 |        0 |    4.52267
_Betweenness2                  |        19116.27 |        19025.63 |    90.64 |     99.52% |  0.0439% |    0.0000% |  3.1837% |        2 |        2 |        1 |        0 |    8.24166
_Betweenness3                  |        19299.00 |        19214.00 |    85.00 |     99.56% |  0.0295% |    0.0000% |  2.7402% |        1 |        1 |        0 |        0 |   12.25564
_Betweenness4                  |        20938.18 |        20853.18 |    85.00 |     99.59% |  0.0206% |    0.0000% |  2.7402% |        1 |        1 |        0 |        0 |   16.16196
Somerville_Bus_Subway_Geometri |          709.44 |          713.14 |    -3.69 |    100.52% |  0.5027% |    0.1954% |  3.9604% |      149 |       62 |        3 |        0 |   19.41082
Somerville_Bus_Subway          |          740.19 |          743.95 |    -3.76 |    100.51% |  0.5074% |    0.2539% |  3.9604% |      173 |       39 |        3 |        0 |   38.43126
Somerville_Homes_Subway        |       377967.80 |       389383.31 | -11415.51 |    102.93% |     inf% |    3.8514% |     inf% |     2579 |     2265 |     1591 |     1001 |  264.22996




# Vectorized betweenness, fix for unrelevant destinations ruining graph
test name                      | madina_flow_sum | Rhino_flow_sum  | sum_diff | sum_smlr_% | sle_mean | sle_median | sle_max  | sle>0.1% | sle>1.0% | sle>3.0% | sle>5.0% | time_spent
_Betweenness1                  |          704.85 |          702.07 |     2.78 |     99.60% |  0.0376% |    0.0000% |  2.4893% |        2 |        2 |        0 |        0 |    3.03662
_Betweenness2                  |        19116.27 |        19025.63 |    90.64 |     99.52% |  0.0439% |    0.0000% |  3.1837% |        2 |        2 |        1 |        0 |    5.43066
_Betweenness3                  |        19299.00 |        19214.00 |    85.00 |     99.56% |  0.0295% |    0.0000% |  2.7402% |        1 |        1 |        0 |        0 |    7.75261
_Betweenness4                  |        20938.18 |        20853.18 |    85.00 |     99.59% |  0.0206% |    0.0000% |  2.7402% |        1 |        1 |        0 |        0 |   10.10918
Somerville_Bus_Subway_Geometri |          700.29 |          713.14 |   -12.84 |    101.80% |  2.3488% |    2.2893% |  5.0852% |      266 |      232 |       96 |        2 |   14.95809
Somerville_Bus_Subway          |          730.95 |          743.95 |   -13.00 |    101.75% |  2.4117% |    2.2917% |  5.1912% |      285 |      251 |      103 |        8 |   30.91729
Somerville_Homes_Subway        |       368173.65 |       389383.31 | -21209.66 |    105.45% |     inf% |    7.2526% |     inf% |     2665 |     2631 |     2521 |     2216 |  325.34906






# SIngle add SIngle remove
test name                      | madina_flow_sum | Rhino_flow_sum  | sum_diff | sum_smlr_% | sle_mean | sle_median | sle_max  | sle>0.1% | sle>1.0% | sle>3.0% | sle>5.0% | time_spent
_Betweenness1                  |          704.85 |          702.07 |     2.78 |     99.60% |  0.0376% |    0.0000% |  2.4893% |        2 |        2 |        0 |        0 |    3.28500
_Betweenness2                  |        19116.27 |        19025.63 |    90.64 |     99.52% |  0.0439% |    0.0000% |  3.1837% |        2 |        2 |        1 |        0 |    6.64996
_Betweenness3                  |        19299.00 |        19214.00 |    85.00 |     99.56% |  0.0295% |    0.0000% |  2.7402% |        1 |        1 |        0 |        0 |    9.48671
_Betweenness4                  |        20938.18 |        20853.18 |    85.00 |     99.59% |  0.0206% |    0.0000% |  2.7402% |        1 |        1 |        0 |        0 |   12.15137
Somerville_Bus_Subway_Geometri |          700.29 |          713.14 |   -12.84 |    101.80% |  2.3488% |    2.2893% |  5.0852% |      266 |      232 |       96 |        2 |   17.88752
Somerville_Bus_Subway          |          730.95 |          743.95 |   -13.00 |    101.75% |  2.4117% |    2.2917% |  5.1912% |      285 |      251 |      103 |        8 |   36.95279
Somerville_Homes_Subway        |       368132.55 |       389383.31 | -21250.76 |    105.46% |     inf% |    7.0262% |     inf% |     2665 |     2619 |     2487 |     2091 |  295.88396





# Classical update light graph
test name                      | madina_flow_sum | Rhino_flow_sum  | sum_diff | sum_smlr_% | sle_mean | sle_median | sle_max  | sle>0.1% | sle>1.0% | sle>3.0% | sle>5.0% | time_spent
_Betweenness1                  |          704.85 |          702.07 |     2.78 |     99.60% |  0.0376% |    0.0000% |  2.4893% |        2 |        2 |        0 |        0 |    3.66067
_Betweenness2                  |        19116.27 |        19025.63 |    90.64 |     99.52% |  0.0439% |    0.0000% |  3.1837% |        2 |        2 |        1 |        0 |    6.61123
_Betweenness3                  |        19299.00 |        19214.00 |    85.00 |     99.56% |  0.0295% |    0.0000% |  2.7402% |        1 |        1 |        0 |        0 |    9.72403
_Betweenness4                  |        20938.18 |        20853.18 |    85.00 |     99.59% |  0.0206% |    0.0000% |  2.7402% |        1 |        1 |        0 |        0 |   12.61041
Somerville_Bus_Subway_Geometri |          700.29 |          713.14 |   -12.84 |    101.80% |  2.3488% |    2.2893% |  5.0852% |      266 |      232 |       96 |        2 |   15.90220
Somerville_Bus_Subway          |          730.95 |          743.95 |   -13.00 |    101.75% |  2.4117% |    2.2917% |  5.1912% |      285 |      251 |      103 |        8 |   32.07222
Somerville_Homes_Subway        |       368132.55 |       389383.31 | -21250.76 |    105.46% |     inf% |    7.0262% |     inf% |     2665 |     2619 |     2487 |     2091 |  363.67400

# paralell_betweenness_exposure (before four fixes...)
test name                      | madina_flow_sum | Rhino_flow_sum  | sum_diff | sum_smlr_% | sle_mean | sle_median | sle_max  | sle>0.1% | sle>1.0% | sle>3.0% | sle>5.0% | time_spent
_Betweenness1                  |          704.85 |          702.07 |     2.78 |     99.60% |  0.0376% |    0.0000% |  2.4893% |        2 |        2 |        0 |        0 |    4.05741
_Betweenness2                  |        19116.27 |        19025.63 |    90.64 |     99.52% |  0.0439% |    0.0000% |  3.1837% |        2 |        2 |        1 |        0 |    7.86811
_Betweenness3                  |        19299.00 |        19214.00 |    85.00 |     99.56% |  0.0295% |    0.0000% |  2.7402% |        1 |        1 |        0 |        0 |   11.35870
_Betweenness4                  |        20938.18 |        20853.18 |    85.00 |     99.59% |  0.0206% |    0.0000% |  2.7402% |        1 |        1 |        0 |        0 |   15.43956
Somerville_Bus_Subway_Geometri |          700.29 |          713.14 |   -12.84 |    101.80% |  2.3488% |    2.2893% |  5.0852% |      266 |      232 |       96 |        2 |   20.17142
Somerville_Bus_Subway          |          730.95 |          743.95 |   -13.00 |    101.75% |  2.4117% |    2.2917% |  5.1912% |      285 |      251 |      103 |        8 |   40.65643

# paralell_betweenness_exposure (after 5 fixes.....)
test name                      | madina_flow_sum | Rhino_flow_sum  | sum_diff | sum_smlr_% | sle_mean | sle_median | sle_max  | sle>0.1% | sle>1.0% | sle>3.0% | sle>5.0% | time_spent
_Betweenness1                  |          704.85 |          702.07 |     2.78 |     99.60% |  0.0376% |    0.0000% |  2.4893% |        2 |        2 |        0 |        0 |    3.85227
_Betweenness2                  |        19116.27 |        19025.63 |    90.64 |     99.52% |  0.0439% |    0.0000% |  3.1837% |        2 |        2 |        1 |        0 |    7.25895
_Betweenness3                  |        19299.00 |        19214.00 |    85.00 |     99.56% |  0.0295% |    0.0000% |  2.7402% |        1 |        1 |        0 |        0 |   10.78336
_Betweenness4                  |        20938.18 |        20853.18 |    85.00 |     99.59% |  0.0206% |    0.0000% |  2.7402% |        1 |        1 |        0 |        0 |   14.21737
Somerville_Bus_Subway_Geometri |          700.29 |          713.14 |   -12.84 |    101.80% |  2.3488% |    2.2893% |  5.0852% |      266 |      232 |       96 |        2 |   18.31064
Somerville_Bus_Subway          |          730.95 |          743.95 |   -13.00 |    101.75% |  2.4117% |    2.2917% |  5.1912% |      285 |      251 |      103 |        8 |   36.99389
Somerville_Homes_Subway        |       368132.55 |       389383.31 | -21250.76 |    105.46% |     inf% |    7.0262% |     inf% |     2665 |     2619 |     2487 |     2091 |  429.40144



### parallel_betweenness
test name                      | madina_flow_sum | Rhino_flow_sum  | sum_diff | sum_smlr_% | sle_mean | sle_median | sle_max  | sle>0.1% | sle>1.0% | sle>3.0% | sle>5.0%
_Betweenness1                  |          704.85 |          702.07 |     2.78 |     99.60% |  0.0376% |    0.0000% |  2.4893% |        2 |        2 |        0 |        0
_Betweenness2                  |        19116.27 |        19025.63 |    90.64 |     99.52% |  0.0439% |    0.0000% |  3.1837% |        2 |        2 |        1 |        0
_Betweenness3                  |        19299.00 |        19214.00 |    85.00 |     99.56% |  0.0295% |    0.0000% |  2.7402% |        1 |        1 |        0 |        0
_Betweenness4                  |        20938.18 |        20853.18 |    85.00 |     99.59% |  0.0206% |    0.0000% |  2.7402% |        1 |        1 |        0 |        0
Somerville_Bus_Subway_Geometri |          709.44 |          713.14 |    -3.69 |    100.52% |  0.5027% |    0.1954% |  3.9604% |      149 |       62 |        3 |        0
Somerville_Bus_Subway          |          740.19 |          743.95 |    -3.76 |    100.51% |  0.5074% |    0.2539% |  3.9604% |      173 |       39 |        3 |        0
Somerville_Homes_Subway        |       377922.37 |       389383.31 | -11460.94 |    102.94% |     inf% |    3.8409% |     inf% |     2578 |     2232 |     1591 |      997
Somerville_Jobs_Subway         |       146309.13 |       149908.30 | -3599.17 |    102.40% |     inf% |    3.5776% |     inf% |     2078 |     1741 |     1286 |      824


In [ ]:
def profile_new():
    betweenness_exposure(
        harvard_square,
        origins=origin_gdf.iloc[:],
        search_radius=test_config.at[test_idx,'Radius'],
        detour_ratio=test_config.at[test_idx,'Detour'],
        decay=False if test_config.at[test_idx,'Elastic_Weights'] else test_config.at[test_idx,'Decay'],
        beta=test_config.at[test_idx,'Beta'],
        decay_method=test_config.at[test_idx,'Decay_Mode'],
        path_detour_penalty='equal',
        closest_destination=test_config.at[test_idx,'Closest_Destination'],
        elastic_weight=test_config.at[test_idx,'Elastic_Weights'],
        turn_penalty=test_config.at[test_idx,'Turns'],
        path_exposure_attribute=None,
        return_path_record=False, 
        destniation_cap=None
    )
    return


def profile_baseline():
    one_betweenness_2(
        network=harvard_square.network,
        search_radius=test_config.at[test_idx,'Radius'],
        origins=origin_gdf,
        detour_ratio=test_config.at[test_idx,'Detour'],
        decay=False if test_config.at[test_idx,'Elastic_Weights'] else test_config.at[test_idx,'Decay'],
        beta=test_config.at[test_idx,'Beta'],
        decay_method=test_config.at[test_idx,'Decay_Mode'],
        path_detour_penalty="equal",  # "exponent" | "power"
        origin_weights=True,
        origin_weight_attribute=None,
        closest_destination=test_config.at[test_idx,'Closest_Destination'],
        destination_weights=True,
        destination_weight_attribute=None,
        perceived_distance=False,
        light_graph=True,
        turn_penalty=test_config.at[test_idx,'Turns'],
        retained_d_idxs=None,
        retained_paths=None,
        retained_distances=None,
        rertain_expensive_data=False
)


In [ ]:
import sys
import geopandas as gpd
import shapely as shp
import networkx as nx
import pandas as pd
import pydeck as pdk

print (sys.version)
print(f"{gpd.__version__ = }")
print(f"{shp.__version__ = }")
print(f"{nx.__version__ = }")
print(f"{pd.__version__ = }")
print(f"{pdk.__version__ = }")

In [ ]:
%load_ext line_profiler
def profile_new():

    betweenness_exposure(
        harvard_square,
        origins=origin_gdf,
        search_radius=test_config.at[test_idx,'Radius'],
        detour_ratio=test_config.at[test_idx,'Detour'],
        decay=False if test_config.at[test_idx,'Elastic_Weights'] else test_config.at[test_idx,'Decay'],
        beta=test_config.at[test_idx,'Beta'],
        decay_method=test_config.at[test_idx,'Decay_Mode'],
        path_detour_penalty='equal',
        closest_destination=test_config.at[test_idx,'Closest_Destination'],
        elastic_weight=test_config.at[test_idx,'Elastic_Weights'],
        turn_penalty=test_config.at[test_idx,'Turns'],
        path_exposure_attribute=None,
        return_path_record=False, 
        destniation_cap=None
    )
    return


def profile_baseline():
    one_betweenness_2(
        network=harvard_square.network,
        search_radius=test_config.at[test_idx,'Radius'],
        origins=origin_gdf,
        detour_ratio=test_config.at[test_idx,'Detour'],
        decay=False if test_config.at[test_idx,'Elastic_Weights'] else test_config.at[test_idx,'Decay'],
        beta=test_config.at[test_idx,'Beta'],
        decay_method=test_config.at[test_idx,'Decay_Mode'],
        path_detour_penalty="equal",  # "exponent" | "power"
        origin_weights=True,
        origin_weight_attribute=None,
        closest_destination=test_config.at[test_idx,'Closest_Destination'],
        destination_weights=True,
        destination_weight_attribute=None,
        perceived_distance=False,
        light_graph=True,
        turn_penalty=test_config.at[test_idx,'Turns'],
        retained_d_idxs=None,
        retained_paths=None,
        retained_distances=None,
        rertain_expensive_data=False
)
    

In [ ]:
%prun profile_baseline()

In [ ]:
%prun profile_new()

In [ ]:
streets = harvard_square.layers['streets'].gdf
path_record = return_dict['path_record']

harvard_square.create_map(
    [
        {'gdf': streets, 'color': [100, 100, 100], 'opacity': 0.1},
        {'gdf': path_record[path_record['origin_id'] == 7796], 'color_by_attribute': 'path_id', 'color_method': 'categorical', 'opacity': 0.1, 'text': 'path_betweenness'},
        {'gdf': origin_layer, 'color': [100, 0, 255]},
        {'gdf': origin_nodes, 'color': [100, 0, 255]},
        {'gdf': origin_joined[['geometry']], 'color': [100, 0, 255]},
        {'gdf': destination_layer, 'color': [255, 0, 100]},
        {'gdf': destination_joined[['geometry']], 'color': [255, 0, 100]},
        {'gdf': destination_nodes, 'color': [255, 0, 100]},
    ],
    #save_as="Test Cases\\" + test_case + '\\'  + test_name + "._difference_map.html"
)

# Update Light Graph

In [ ]:
def update_light_graph(self, graph: nx.Graph, add_nodes: list = [], remove_nodes: list = []):
    """
    Updates the given graph object by adding nodes to and removing nodes from it.

    Args:
        graph: The given networkx Graph object to be edited
        add_nodes: a list of nodes to be added
        remove_nodes: a list of nodes to be removed

    Returns:
        none
    """
    
    if "added_nodes" not in graph.graph:
        graph.graph["added_nodes"] = []

    # Add nodes
    if len(add_nodes) > 0:

        existing_nodes = graph.graph["added_nodes"]
        for node_idx in add_nodes:
            if node_idx in existing_nodes:
                print(f'node ({node_idx}) is already added...{add_nodes = }\t{existing_nodes = }')
                print(graph)
                add_nodes.remove(node_idx)
            else:
                graph.graph["added_nodes"].append(node_idx)

        edge_nodes = {}
        for key, value in self.nodes.loc[graph.graph["added_nodes"]].groupby("nearest_edge_id"):
            edge_nodes[int(key)] = list(value.index)
            
        for edge_id in edge_nodes:
            neighbors = edge_nodes[edge_id]
            insert_neighbors = set(add_nodes).intersection(set(neighbors))
            existing_neighbors = set(neighbors) - insert_neighbors

            if len(insert_neighbors) == 0:
                continue
                #self.nodes.at[node_idx, ""]
            if len(neighbors) == 1:
                node_idx = neighbors[0]
                graph.add_edge(
                    int(self.nodes.at[node_idx, "edge_end_node"]),
                    int(node_idx),
                    weight=max(self.nodes.at[node_idx, "weight_to_end"], 0),
                    id=edge_id
                )
                graph.add_edge(
                    int(node_idx),
                    int(self.nodes.at[node_idx, "edge_start_node"]),
                    weight=max(self.nodes.at[node_idx, 'weight_to_start'], 0),
                    id=edge_id
                )
                graph.remove_edge(self.nodes.at[node_idx, "edge_end_node"], int(self.nodes.at[node_idx, "edge_start_node"]))
                
            else:
                # start a chain addition of neighbors, starting from the 'left',
                # so, need to sort based on distance from left
                segment_weight = self.edges.at[edge_id, "weight"]

                chain_start = self.nodes.at[neighbors[0], "edge_end_node"]
                chain_end = self.nodes.at[neighbors[0], "edge_start_node"]

                chain_distances = [self.nodes.at[node, "weight_to_end"] for node in neighbors]

                if len(existing_neighbors) == 0:  # if there are no existing neighbors, remove the original edge
                    graph.remove_edge(int(chain_start), int(chain_end))
                else:  # if there are existing neighbors, remove them. This would also remove their associated edges
                    for node in existing_neighbors:
                        graph.remove_node(node)

                chain_nodes = [chain_start] + neighbors + [chain_end]
                chain_distances = [0] + chain_distances + [segment_weight]

                chain_nodes = np.array(chain_nodes)
                chain_distances = np.array(chain_distances)
                sorting_index = np.argsort(chain_distances)

                # np arrays allows calling them by a list index
                chain_nodes = chain_nodes[sorting_index]
                chain_distances = chain_distances[sorting_index]
                # print(f"{chain_nodes = }\t{chain_distances}")
                accumilated_weight = 0
                for seq in range(len(chain_nodes) - 1):
                    graph.add_edge(
                        int(chain_nodes[seq]),
                        int(chain_nodes[seq + 1]),
                        # TODO: change this to either defaults to distance or a specified column for a weight..
                        weight=max(chain_distances[seq + 1] - chain_distances[seq], 0),
                        ##avoiding small negative numbers due to numerical error when two nodes are superimposed.
                        id=edge_id
                    )
                    accumilated_weight += (chain_distances[seq + 1] - chain_distances[seq])
    
    # Removing nodes
    for node_idx in remove_nodes:
        node_idx = int(node_idx)
        if node_idx not in graph.nodes:
            print(f"attempting to remove node {node_idx} that's not in graph {str(graph)}")
            continue

        if len(graph.adj[node_idx]) != 2:
            print(f"attempting to remove a node {node_idx = } that's not degree 2, adjacent to: {graph.adj[node_idx]}")
            continue

        neighbors = list(graph.adj[node_idx])

        start = int(neighbors[0])
        end = int(neighbors[1])
        weight = graph.adj[node_idx][start]["weight"] \
                + graph.adj[node_idx][end]["weight"]

        original_edge_id = self.nodes.at[node_idx, "nearest_edge_id"]

        # remove node after we got the attributes we needed..
        graph.remove_node(node_idx)
        graph.graph["added_nodes"].remove(node_idx)
        graph.add_edge(
            start,
            end,
            weight=weight,
            id=original_edge_id
        )
        
    return graph

In [ ]:
for origin_idx in origin_gdf.index:
    start = time.time()
    update_light_graph(
        harvard_square.network,
        harvard_square.network.d_graph,
        add_nodes=[origin_idx],
        #remove_nodes=[origin_idx]
    )

    update_light_graph(
        harvard_square.network,
        harvard_square.network.d_graph,
        #add_nodes=[origin_idx],
        remove_nodes=[origin_idx]
    )
    

# COnstructing the Manhattan Case..
## Don't forget to check for node insertion, set  return_all=False in the spatial index

In [ ]:
import time
start = time.time()

buildings_file = r"C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\madina\unit_testing\Test Cases\Manhattan\Home_PT_6538.geojson"
subway_file = r"C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\madina\unit_testing\Test Cases\Manhattan\Metro_PT_6538.geojson"
network_file = r"C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\madina\unit_testing\Test Cases\Manhattan\network_clipped_dupremovedAS.geojson"

import sys
sys.path.append('../')
from madina.zonal.zonal import Zonal


harvard_square = Zonal()

print(f"{(time.time()-start)*1000:6.2f}ms\t imports done, object created")
start = time.time()

#harvard_square.load_layer(
#    layer_name='streets',
#    file_path=network_file
#    )


print(f"{(time.time()-start)*1000:6.2f}ms\t street data loaded")
start = time.time()

harvard_square.load_layer(
    layer_name='buildings',
    file_path=buildings_file
    )

buildings_gdf = harvard_square.layers['buildings'].gdf
harvard_square.layers['buildings'].gdf = buildings_gdf[~buildings_gdf['FID'].isin([27967, 9140, 3974])]

print(f"{(time.time()-start)*1000:6.2f}ms\t building data loaded")
start = time.time()

harvard_square.load_layer(
    layer_name='subway',
    file_path=subway_file
    )

print(f"{(time.time()-start)*1000:6.2f}ms\t subway data loaded")

In [ ]:
start = time.time()

harvard_square.create_street_network(
    source_layer='streets', 
    discard_redundant_edges=False, 
    node_snapping_tolerance=0
)

print(f"{(time.time()-start)*1000:6.2f}ms\t street network created")
start = time.time()

harvard_square.insert_node(
    layer_name='buildings', 
    label='origin', 
    weight_attribute='TotalPop'
)


print(f"{(time.time()-start)*1000:6.2f}ms\t origins insertes")
start = time.time()

harvard_square.insert_node(
    layer_name='subway', 
    label='destination', 
    weight_attribute='line_ent_st'
)

print(f"{(time.time()-start)*1000:6.2f}ms\t destinations insertes")


start = time.time()

harvard_square.create_graph(light_graph=True, d_graph=True)

print(f"{(time.time()-start)*1000:6.2f}ms\t graph created insertes")
start = time.time()

return_dict = parallel_betweenness(
    harvard_square.network,
    search_radius=800,
    detour_ratio=1.15,
    decay=False,
    decay_method='exponent',  # "power", "exponent"
    beta=0.004,
    path_detour_penalty="equal",  # "power", "exponent", "equal"
    origin_weights=True,
    closest_destination=False,
    destination_weights=True, 
    # perceived_distance=False,
    num_cores=8,
    light_graph=True,
    turn_penalty=False,
)
simulated_sum_of_flow = return_dict['edge_gdf']['betweenness'].sum()

print(f"{(time.time()-start)*1000:6.2f}ms\t Betweenness estimated")
start = time.time()

In [ ]:
joined_results = harvard_square.layers['streets'].gdf.join(harvard_square.network.edges[['parent_street_id', 'betweenness']].set_index('parent_street_id'))
joined_results[['__GUID', 'betweenness', 'geometry']].to_csv('2023-07-07 manhattan betweenness flow test.csv')
joined_results[['__GUID', 'betweenness', 'geometry']].to_file('2023-07-07 manhattan betweenness flow test.geoJSON', driver="GeoJSON")

In [ ]:
harvard_square.create_map(
    [
        #{
            #'layer': 'streets',
            #'color': [125, 125, 125],
        #},
        {
            'gdf': harvard_square.layers['homes'].gdf,
            'color_by_attribute': 'line_ent_st',
            "color_method": 'gradient'
        }
    ],
    save_as="map.html"
)